# Importing Libraries

In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import numpy as np
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
import copy

# Preprocessing

In [ ]:
def load_subdata(data):
    X1 = []
    y1 = []

    #X2 = []
    #y2 = []
    lbl = 0
    dic = {}
    for folder_name in os.listdir(data):
        
        print(folder_name)
        Label = lbl
        dic[folder_name] = Label
        

        count_imag = 0;

        for filename in tqdm(os.listdir(data + '/' + folder_name)):
            image = copy.deepcopy(np.array(Image.open(data +'/'+ folder_name + '/' + filename),dtype=np.uint8))
            X1.append(image)
            y1.append(Label) 
        lbl +=1   

             
    X1 = np.asarray(X1)
    y1 = np.asarray(y1)
    #X2 = np.asarray(X2)
    #y2 = np.asarray(y2)
    return X1,y1, dic

X_train, y_train, lanel_dic = load_subdata(r'../input/flowers-img-classification/train')


In [ ]:
print(lanel_dic)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_val, y_val, lanel_val_dic = load_subdata(r'../input/flowers-img-classification/val')

In [ ]:
print(X_val.shape)
print(y_val.shape)

In [ ]:
from keras.utils.np_utils import to_categorical

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

In [ ]:
y_trainHot = np.uint8(to_categorical(y_train, num_classes = 104))
y_valHot = np.uint8(to_categorical(y_val, num_classes = 104))

In [ ]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [ ]:
X_train_s, y_train_s = shuffle_in_unison(X_train, y_trainHot)
X_val_s, y_val_s = shuffle_in_unison(X_val, y_valHot)

In [ ]:
print(X_train_s.shape)
print(y_train_s.shape)

print(X_val_s.shape)
print(y_val_s.shape)


# Model Training

In [ ]:
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet201

In [ ]:
IMG_SIZE =224
NUM_CLASSES = 104
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs

#     VGG Model

In [ ]:
def VGGupdated(input_tensor=None, classes=2):
    img_rows, img_cols = 224,224  # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)

    img_input = Input(shape=img_dim)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.

    model = Model(inputs=img_input, outputs=x, name='VGGdemo')

    return model

model = VGGupdated(classes = 104)

model.compile(optimizer=optimizers.Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_s,y_train_s, epochs=15, validation_data = (X_val_s, y_val_s), shuffle = True, batch_size=8, steps_per_epoch=2000)
model.save('VGG_model.h5')

In [ ]:
tf.keras.models.save_model(model,'VGG_model.hdf5')

# EfficientNet Model

In [ ]:
outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(x)
eff_model = tf.keras.Model(inputs, outputs)
eff_model.compile(
      optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
  )

eff_model.summary()
epochs = 10  # @param {type: "slider", min:10, max:100}
#hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

In [ ]:
eff_model.fit(X_train_s,y_train_s, epochs=20, validation_data = (X_val_s, y_val_s), shuffle = True, batch_size=8, steps_per_epoch=2000)
eff_model.save('eff_model.h5')

In [ ]:
tf.keras.models.save_model(eff_model,'eff_model_1.hdf5')

#    MobileNet Model

In [ ]:
outputs = MobileNetV2(include_top=True, weights=None, classes=NUM_CLASSES)(x)
MobileNetV2_model = tf.keras.Model(inputs, outputs)
MobileNetV2_model.compile(
      optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
  )

MobileNetV2_model.summary()
epochs = 10  # @param {type: "slider", min:10, max:100}
#hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

In [ ]:
MobileNetV2_model.fit(X_train_s,y_train_s, epochs=25, validation_data = (X_val_s, y_val_s), shuffle = True, batch_size=8, steps_per_epoch=2000)
MobileNetV2_model.save('MobileNet_model.h5')

In [ ]:
tf.keras.models.save_model(MobileNetV2_model,'MobileNet_model.hdf5')

# ResNet Model

In [ ]:
outputs = ResNet50V2(include_top=True, weights=None, classes=NUM_CLASSES)(x)
res_model = tf.keras.Model(inputs, outputs)
res_model.compile(
      optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
  )

res_model.summary()
epochs = 10  # @param {type: "slider", min:10, max:100}
#hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

In [ ]:
res_model.fit(X_train_s,y_train_s, epochs=20, validation_data = (X_val_s, y_val_s), shuffle = True, batch_size=8, steps_per_epoch=2000)
res_model.save('res_model.h5')

In [ ]:
tf.keras.models.save_model(res_model,'Res_model.hdf5')

# DenseNet Model

In [ ]:
outputs = DenseNet201(include_top=True, weights=None, classes=NUM_CLASSES)(x)
DenseNet201_model = tf.keras.Model(inputs, outputs)
DenseNet201_model.compile(
      optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
  )

DenseNet201_model.summary()
epochs = 10  # @param {type: "slider", min:10, max:100}
#hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

In [ ]:
DenseNet201_model.fit(X_train_s,y_train_s, epochs=20, validation_data = (X_val_s, y_val_s), shuffle = True, batch_size=8, steps_per_epoch=2000)
DenseNet201_model.save('DenseNet_model.h5')

In [ ]:
tf.keras.models.save_model(DenseNet201_model,'DenseNet_model.hdf5')

# Load Model

In [ ]:
Eff_model = keras.models.load_model('../input/efficientnet-model/eff_model (1).h5')

# Validating Model Output

In [ ]:
import PIL
def validate_set(img):

    X_valid = []

    
    image = Image.open(img)
        #image = ImageOps.grayscale(image)
        
    image = np.array(image)
    image_data_as_arr = np.asarray(image)
        
    X_valid.append(image_data_as_arr)
    X_valid = np.asarray(X_valid)   
    X_valid = tf.expand_dims(X_valid, axis=-1)
    return X_valid

X_val = validate_set(r'../input/flowers-img-classification/test/0087eeefc.jpeg')


y_pred = Eff_model.predict(X_val)
Y_pred_classes = np.argmax(y_pred,axis=1)
print(Y_pred_classes)
plt.figure(figsize=(64, 64))
img = PIL.Image.open(r'../input/flowers-img-classification/test/0087eeefc.jpeg')
plt.axis('off')


keys = [k for k, v in lanel_dic.items() if v == Y_pred_classes]
print(keys)
plt.imshow(img)